<a href="https://colab.research.google.com/github/tonytoon/ivylearnnotebooks/blob/main/module_import_to_subaccount.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a sample script to copy modules from one course to all courses for a given term in a given subaccount. This is used frequently for one of our service
areas to create a module filled with resources specific to them, then push it out only to their courses.

With modification, this could be used to push multiple types of content out. You'll need to play with the "select" argument in the content migration.

In [ ]:
!pip install canvasapi

In [ ]:
from canvasapi import Canvas # used for interfacing with canvas

# Canvas API URL
# ALWAYS test scripts on the beta server until you are 100% that it will work properly.
#API_URL = ".beta.instructure.com" # beta
API_URL = ".instructure.com" # prod

# Canvas API key
#API_KEY = "" #beta
API_KEY = "" #prod

# Initialize a new Canvas object
canvas = Canvas(API_URL, API_KEY)

In [ ]:
# define which term we're interested in, which course we're copying from, and
# which subaccount we're copying into.
term = 3265 # summer 2022
source_course_id = 1138574 #
target_subaccount = 16166 #

In [ ]:

# shouldn't need to modify anything below here.
# unless you want to change what's imported

mods = [] # empty list of module ids to populate later

account = canvas.get_account(target_subaccount)

# get the id of all modules in the source course
modules = canvas.get_course(source_course_id).get_modules()
for m in modules:
  mods.append(m.id)

# get all courses for a given term in a given subaccount
courses = account.get_courses(enrollment_term_id=term)

# create our content migration.
# see: https://canvasapi.readthedocs.io/en/stable/course-ref.html#canvasapi.course.Course.create_content_migration
# see: https://canvas.instructure.com/doc/api/content_migrations.html

for course in courses:
    print(course)
    course.create_content_migration(
      "course_copy_importer",
      settings = {
          "source_course_id" : source_course_id,
          "importer_skips" : "all_course_settings"
      },
      
      # select expects a dict which has a key defining what you want copied
      # (modules in our case) and a value which is a list of IDs to copy.
      # I was hoping to be able to tell it to just copy all modules,
      # but couldn't figure out a way so I go through the source course
      # and collect all the module ids beforehand.
      
      select = {
          "modules" : mods
      }
    )